# DATA

## Data Preprocess and Save

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

train_transaction = pd.read_csv('/content/gdrive/MyDrive/inzva_week2_hw_dataset/train_transaction.csv')
test_transaction = pd.read_csv('/content/gdrive/MyDrive/inzva_week2_hw_dataset/test_transaction.csv')
train_identity = pd.read_csv('/content/gdrive/MyDrive/inzva_week2_hw_dataset/train_identity.csv')
test_identity = pd.read_csv('/content/gdrive/MyDrive/inzva_week2_hw_dataset/test_identity.csv')

In [3]:
train_transaction_identity = train_transaction.merge(train_identity, on='TransactionID',how='left')

del train_transaction
del train_identity

test_transaction_identity = test_transaction.merge(test_identity, on='TransactionID',how='left')

del test_transaction
del test_identity

In [4]:
y_train_transaction_identity = train_transaction_identity['isFraud']

In [5]:
train_transaction_identity.drop('isFraud', axis=1, inplace=True)

In [6]:
train_transaction_identity.shape

(590540, 433)

In [7]:
import numpy as np
null_percent = pd.DataFrame((np.sum(pd.isnull(train_transaction_identity)).sort_values(ascending=False)/len(train_transaction_identity))*100, columns=[ 'null_percent'])
null_percent

,null_percent
id_24,99.196159
id_25,99.130965
id_07,99.127070
id_08,99.127070
id_21,99.126393
...,...
C11,0.000000
C12,0.000000
C13,0.000000
C14,0.000000


In [8]:
null_percent.reset_index(inplace=True)

In [9]:
def drop_cols(threshold):
  tmp_df = pd.DataFrame()
  tmp_df = null_percent[null_percent['null_percent'] > threshold]
  
  for col in tmp_df['index']:
    train_transaction_identity.drop(columns= col, axis = 1, inplace= True)

    if "id" in col: 
      test_transaction_identity.drop(columns= col.replace('_', '-'), axis = 1, inplace= True)
    else:
      test_transaction_identity.drop(columns= col , axis = 1, inplace= True)

drop_cols(30) # yüzde 30'un üzerinde satırı null olan kolonları at

In [10]:
print(train_transaction_identity.shape)
print(test_transaction_identity.shape)

(590540, 201)
(506691, 201)


In [11]:
#train_transaction_identity.columns = ["col"+str(i) for i in range(201)]
#test_transaction_identity.columns = ["col"+str(i) for i in range(201)]

In [12]:
frames = [train_transaction_identity, test_transaction_identity]

In [13]:
DATA = pd.concat(frames)

del train_transaction_identity
del test_transaction_identity

In [14]:
DATA.shape

(1097231, 201)

In [15]:
DATA.fillna(DATA.mean(), inplace=True)

In [16]:
DATA.count()

TransactionID     1097231
TransactionDT     1097231
TransactionAmt    1097231
ProductCD         1097231
card1             1097231
                   ...   
V317              1097231
V318              1097231
V319              1097231
V320              1097231
V321              1097231
Length: 201, dtype: int64

In [17]:
DATA['card4'].fillna(DATA['card4'].mode()[0], inplace=True)
DATA['card6'].fillna(DATA['card6'].mode()[0], inplace=True)
DATA['P_emaildomain'].fillna(DATA['P_emaildomain'].mode()[0], inplace=True)
DATA['M6'].fillna(DATA['M6'].mode()[0], inplace=True)

In [18]:
DATA.drop(['TransactionID'], axis=1, inplace=True)

In [19]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if df[feature_name].dtype == 'O':
          pass
        else:
          max_value = df[feature_name].max()
          min_value = df[feature_name].min()
          result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

DATA = normalize(DATA)

In [20]:
DATA

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D4,D10,D15,M6,V12,V13,V14,V15,V16,V17,V18,V19,V20,...,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,0.000000e+00,0.002144,W,0.743002,0.5262,0.378788,discover,0.306569,credit,0.488636,0.836957,gmail.com,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.001748,0.000000,0.000627,0.000000,0.000343,0.0007,0.021841,0.230586,0.011916,0.070698,T,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,...,0.015873,0.014706,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000778,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.000000,0.0,0.00000,0.000122,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000122,0.000000,0.0,0.0,0.0
1,2.930150e-08,0.000907,W,0.100879,0.6080,0.378788,mastercard,0.014599,credit,0.511364,0.836957,gmail.com,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.000000,0.000000,0.000314,0.000000,0.000343,0.0007,0.000000,0.100577,0.000000,0.070698,T,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,...,0.015873,0.014706,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
2,2.021804e-06,0.001847,W,0.210554,0.7800,0.378788,visa,0.481752,debit,0.522727,0.836957,outlook.com,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.001748,0.000000,0.000314,0.000000,0.000343,0.0007,0.000000,0.100577,0.000000,0.339012,F,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,...,0.015873,0.014706,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
3,2.900849e-06,0.001565,W,0.984767,0.9340,0.378788,mastercard,0.124088,debit,0.854545,0.836957,yahoo.com,0.000427,0.000879,0.0,0.000000,0.0,0.001775,0.000000,0.0000,0.001748,0.000000,0.000314,0.000000,0.008568,0.0007,0.174727,0.178071,0.076994,0.165247,F,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,...,0.000000,0.000000,0.0,0.105263,0.0,0.129032,0.0,0.000000,0.0,0.000000,0.0,0.001151,0.029549,0.025862,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.000000,0.0,0.00007,0.001834,0.001287,0.0,0.002117,0.0,0.000807,0.000000,0.000000,0.000000,0.00007,0.001465,0.001099,0.0,0.0,0.0
4,3.105959e-06,0.001565,H,0.201012,0.8280,0.378788,mastercard,0.014599,credit,0.727273,0.836957,gmail.com,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0003,0.000000,0.000307,0.000314,0.000000,0.000343,0.0007,0.000000,0.230586,0.129729,0.228409,F,0.141165,0.099965,0.99967,0.010113,0.005308,0.013043,0.013149,0.064498,0.034652,...,0.015873,0.014706,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.05,0.058824,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,9.999981e-01,0.002964,C,0.737598,0.5500,0.643939,mastercard,0.905109,debit,0.418182,0.543478,gmail.com,0.000213,0.000176,0.0,0.000444,0.0,0.000444,0.000443,0.0000,0.000000,0.000000,0.000314,0.000314,0.000343,0.0007,0.000000,0.100577

In [21]:
pd.options.display.max_rows = 4000

In [22]:
DATA.count()

TransactionDT     1097231
TransactionAmt    1097231
ProductCD         1097231
card1             1097231
card2             1097231
card3             1097231
card4             1097231
card5             1097231
card6             1097231
addr1             1097231
addr2             1097231
P_emaildomain     1097231
C1                1097231
C2                1097231
C3                1097231
C4                1097231
C5                1097231
C6                1097231
C7                1097231
C8                1097231
C9                1097231
C10               1097231
C11               1097231
C12               1097231
C13               1097231
C14               1097231
D1                1097231
D4                1097231
D10               1097231
D15               1097231
M6                1097231
V12               1097231
V13               1097231
V14               1097231
V15               1097231
V16               1097231
V17               1097231
V18               1097231
V19         

In [26]:
DATA.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/DATA_before_onehot.csv")

In [3]:
import pandas as pd
DATA = pd.read_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/DATA_before_onehot.csv")

In [6]:
DATA.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def one_hot_col(df, col):
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(df[col])

  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded_col = onehot_encoder.fit_transform(integer_encoded)

  df_new = pd.DataFrame(data=onehot_encoded_col)
  print(df_new)
  return df_new

In [8]:
df_productCD = one_hot_col(DATA, 'ProductCD')

DATA = pd.concat([DATA, df_productCD], axis=1)
DATA.drop('ProductCD', axis=1, inplace = True)

del df_productCD
DATA.shape

           0    1    2    3    4
0        0.0  0.0  0.0  0.0  1.0
1        0.0  0.0  0.0  0.0  1.0
2        0.0  0.0  0.0  0.0  1.0
3        0.0  0.0  0.0  0.0  1.0
4        0.0  1.0  0.0  0.0  0.0
...      ...  ...  ...  ...  ...
1097226  1.0  0.0  0.0  0.0  0.0
1097227  1.0  0.0  0.0  0.0  0.0
1097228  0.0  0.0  0.0  0.0  1.0
1097229  0.0  0.0  0.0  0.0  1.0
1097230  1.0  0.0  0.0  0.0  0.0

[1097231 rows x 5 columns]


(1097231, 204)

In [9]:
df_card4 = one_hot_col(DATA, 'card4')

DATA = pd.concat([DATA, df_card4], axis=1)
DATA.drop('card4', axis=1, inplace = True)

del df_card4
DATA.shape

           0    1    2    3
0        0.0  1.0  0.0  0.0
1        0.0  0.0  1.0  0.0
2        0.0  0.0  0.0  1.0
3        0.0  0.0  1.0  0.0
4        0.0  0.0  1.0  0.0
...      ...  ...  ...  ...
1097226  0.0  0.0  1.0  0.0
1097227  0.0  0.0  1.0  0.0
1097228  0.0  0.0  0.0  1.0
1097229  0.0  0.0  1.0  0.0
1097230  0.0  0.0  0.0  1.0

[1097231 rows x 4 columns]


(1097231, 207)

In [10]:
df_card6 = one_hot_col(DATA, 'card6')

DATA = pd.concat([DATA, df_card6], axis=1)
DATA.drop('card6', axis=1, inplace = True)

del df_card6
DATA.shape

           0    1    2    3
0        0.0  1.0  0.0  0.0
1        0.0  1.0  0.0  0.0
2        0.0  0.0  1.0  0.0
3        0.0  0.0  1.0  0.0
4        0.0  1.0  0.0  0.0
...      ...  ...  ...  ...
1097226  0.0  0.0  1.0  0.0
1097227  0.0  0.0  1.0  0.0
1097228  0.0  0.0  1.0  0.0
1097229  0.0  0.0  1.0  0.0
1097230  0.0  1.0  0.0  0.0

[1097231 rows x 4 columns]


(1097231, 210)

In [11]:
df_email = one_hot_col(DATA, 'P_emaildomain')

DATA = pd.concat([DATA, df_email], axis=1)
DATA.drop('P_emaildomain', axis=1, inplace = True)

del df_email
DATA.shape

          0    1    2    3    4    5    6   ...   53   54   55   56   57   58   59
0        0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
1097226  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1097227  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1097228  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1097229  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1097230  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[10

(1097231, 269)

In [12]:
df_m6 = one_hot_col(DATA, 'M6')

DATA = pd.concat([DATA, df_m6], axis=1)
DATA.drop('M6', axis=1, inplace = True)

del df_m6
DATA.shape

           0    1
0        0.0  1.0
1        0.0  1.0
2        1.0  0.0
3        1.0  0.0
4        1.0  0.0
...      ...  ...
1097226  1.0  0.0
1097227  1.0  0.0
1097228  1.0  0.0
1097229  1.0  0.0
1097230  1.0  0.0

[1097231 rows x 2 columns]


(1097231, 270)

In [13]:
DATA

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D4,D10,D15,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,...,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,0,1
0,0.000000e+00,0.002144,0.743002,0.5262,0.378788,0.306569,0.488636,0.836957,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.001748,0.000000,0.000627,0.000000,0.000343,0.0007,0.021841,0.230586,0.011916,0.070698,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,0.000000,0.000000,0.066667,0.034483,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.930150e-08,0.000907,0.100879,0.6080,0.378788,0.014599,0.511364,0.836957,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.000000,0.000000,0.000314,0.000000,0.000343,0.0007,0.000000,0.100577,0.000000,0.070698,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,0.000000,0.000000,0.066667,0.034483,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.021804e-06,0.001847,0.210554,0.7800,0.378788,0.481752,0.522727,0.836957,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0000,0.001748,0.000000,0.000314,0.000000,0.000343,0.0007,0.000000,0.100577,0.000000,0.339012,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,0.000000,0.000000,0.066667,0.034483,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.900849e-06,0.001565,0.984767,0.9340,0.378788,0.124088,0.854545,0.836957,0.000427,0.000879,0.0,0.000000,0.0,0.001775,0.000000,0.0000,0.001748,0.000000,0.000314,0.000000,0.008568,0.0007,0.174727,0.178071,0.076994,0.165247,0.250000,0.166667,1.00000,0.000000,0.000000,0.000000,0.000000,0.076923,0.040000,0.000000,0.000000,0.066667,0.034483,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3.105959e-06,0.001565,0.201012,0.8280,0.378788,0.014599,0.727273,0.836957,0.000213,0.000176,0.0,0.000000,0.0,0.000444,0.000000,0.0003,0.000000,0.000307,0.000314,0.000000,0.000343,0.0007,0.000000,0.230586,0.129729,0.228409,0.141165,0.099965,0.99967,0.010113,0.005308,0.013043,0.013149,0.064498,0.034652,0.037706,0.024016,0.069336,0.036677,0.076223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097226,9.999981e-01,0.002964,0.737598,0.5500,0.643939,0.905109,0.418182,0.543478,0.000213,0.000176,0.0,0.000444,0.0,0.000444,0.000443,0.0000,0.000000,0.000000,0.000314,0.000314,0.000343,0.0007,0.000000,0.100577,0.000000,0.070698,0.000000,0.000000,1.00000,0.076923,0.040000,0.000000,0.000000,0.076923,0.040000,0.000000,0.000000,0.066667,0.034483,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1097227,9.999983e-01,0.000381,0.123814,0.6160,0.643939,0.905109,0.434641,0.834406,0.000213,0.000527,0.0,0.000444,0.0,0.000444,0.000443,0.0006,0.000000,0.000614,0.000314,0.000314,0.001028,0.0007,0.024961,0.113768,0.000000,0.084327,0.000000,0.000000,1.00000,0.076923,0.040000,0.066667,

In [14]:
DATA.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/DATA_preprocessed.csv")

In [15]:
DATA.shape

(1097231, 270)

## Split Preprocessed Data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
DATA = pd.read_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/DATA_preprocessed.csv")
DATA.drop('Unnamed: 0', axis=1, inplace=True)
DATA.shape

(1097231, 270)

In [3]:
y_train_data = pd.read_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/Y_train_transaction_identity.csv")
y_train_data.drop('Unnamed: 0', axis=1, inplace=True)
y_train_data.shape

(590540, 1)

In [4]:
x_train_data = DATA.iloc[:590540,:]
x_train_data.shape

(590540, 270)

In [5]:
test_data = DATA.iloc[590540:,:]
test_data.shape

(506691, 270)

# MODELS

## Imports

In [6]:
!pip install scikit-optimize

In [7]:
!pip install catboost

In [8]:
from catboost import Pool, CatBoostClassifier, cv
from numpy import loadtxt
from sklearn.metrics import accuracy_score
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt  
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

In [9]:
from sklearn.model_selection import train_test_split

X_train = pd.DataFrame()
y_train = pd.DataFrame()
X_test = pd.DataFrame()
y_test = pd.DataFrame()

X_train, X_test, y_train, y_test = train_test_split(x_train_data, y_train_data, test_size=0.3)

In [10]:
print(y_test.value_counts())
print(y_train.value_counts())

isFraud
0          170925
1            6237
dtype: int64
isFraud
0          398952
1           14426
dtype: int64


## CatBoost (Default Parameters)
#### Took 0 min
#### 0.72078 Accuracy on Kaggle 
#### 0.978 Accuracy on Test Set

In [14]:
model = CatBoostClassifier(task_type='GPU', eval_metric='F1')  # 0 dk sürdü

model.fit(X_train,y_train,eval_set=(X_test,y_test))

Learning rate set to 0.042671
0:	learn: 0.3536964	test: 0.3606838	best: 0.3606838 (0)	total: 16.9ms	remaining: 16.9s
1:	learn: 0.2904376	test: 0.2916611	best: 0.3606838 (0)	total: 32.4ms	remaining: 16.2s
2:	learn: 0.3052620	test: 0.3104079	best: 0.3606838 (0)	total: 48ms	remaining: 16s
3:	learn: 0.2704297	test: 0.2770740	best: 0.3606838 (0)	total: 63.5ms	remaining: 15.8s
4:	learn: 0.2405962	test: 0.2449833	best: 0.3606838 (0)	total: 77.9ms	remaining: 15.5s
5:	learn: 0.2410508	test: 0.2456336	best: 0.3606838 (0)	total: 92.9ms	remaining: 15.4s
6:	learn: 0.2233551	test: 0.2224429	best: 0.3606838 (0)	total: 107ms	remaining: 15.2s
7:	learn: 0.2326718	test: 0.2363202	best: 0.3606838 (0)	total: 123ms	remaining: 15.2s
8:	learn: 0.2609408	test: 0.2649537	best: 0.3606838 (0)	total: 138ms	remaining: 15.2s
9:	learn: 0.2749320	test: 0.2801365	best: 0.3606838 (0)	total: 153ms	remaining: 15.1s
10:	learn: 0.2738979	test: 0.2790634	best: 0.3606838 (0)	total: 168ms	remaining: 15.1s
11:	learn: 0.2783404	

In [15]:
print("Start Predicting")
predictions = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))
print("Confusion Matrix: \n" ,metrics.confusion_matrix(y_test,predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9783
AUC : 0.915974
F1 Score : 0.568795
Confusion Matrix: 
 [[170796    241]
 [  3595   2530]]


In [16]:
predictions = model.predict(test_data)

prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/catboost_default_parameters_DATA_f1.csv")

In [17]:
model = CatBoostClassifier(task_type='GPU', eval_metric='Accuracy')  # 0 dk sürdü

model.fit(X_train,y_train,eval_set=(X_test,y_test))

Learning rate set to 0.042671
0:	learn: 0.9698000	test: 0.9704451	best: 0.9704451 (0)	total: 17.5ms	remaining: 17.4s
1:	learn: 0.9696815	test: 0.9700331	best: 0.9704451 (0)	total: 31.9ms	remaining: 15.9s
2:	learn: 0.9697855	test: 0.9702306	best: 0.9704451 (0)	total: 48.1ms	remaining: 16s
3:	learn: 0.9694831	test: 0.9700444	best: 0.9704451 (0)	total: 64.3ms	remaining: 16s
4:	learn: 0.9689388	test: 0.9694178	best: 0.9704451 (0)	total: 80.1ms	remaining: 15.9s
5:	learn: 0.9689703	test: 0.9695251	best: 0.9704451 (0)	total: 97.1ms	remaining: 16.1s
6:	learn: 0.9686752	test: 0.9690622	best: 0.9704451 (0)	total: 113ms	remaining: 16s
7:	learn: 0.9688542	test: 0.9693557	best: 0.9704451 (0)	total: 129ms	remaining: 15.9s
8:	learn: 0.9694420	test: 0.9699653	best: 0.9704451 (0)	total: 144ms	remaining: 15.9s
9:	learn: 0.9696960	test: 0.9702363	best: 0.9704451 (0)	total: 161ms	remaining: 15.9s
10:	learn: 0.9697178	test: 0.9702814	best: 0.9704451 (0)	total: 178ms	remaining: 16s
11:	learn: 0.9698315	test

In [18]:
print("Start Predicting")
predictions = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))
print("Confusion Matrix: \n" ,metrics.confusion_matrix(y_test,predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9783
AUC : 0.915958
F1 Score : 0.568667
Confusion Matrix: 
 [[170794    243]
 [  3595   2530]]


In [13]:
predictions = model.predict(test_data)

prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/catboost_default_parameters_DATA.csv")

## CatBoost (Bayesian Search)
#### Took 1 min
#### 0.7286 Accuracy on Kaggle 
#### 0.979 Accuracy on Test Set

In [20]:
# Converting average precision score into a scorer suitable for model selection
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True) # f1 olarak degistirerek de dene # 1 dk sürdü
# Defining your search space
search_spaces = {'iterations': Integer(10, 300),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 #'random_strength': Real(1e-9, 10, 'log-uniform'),
                 #'bagging_temperature': Real(0.0, 1.0),
                 #'border_count': Integer(1, 255),
                 #'l2_leaf_reg': Integer(2, 30),
                 #'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                 }
# Setting up BayesSearchCV
opt = BayesSearchCV(model,
                    search_spaces,
                    scoring=roc_auc,
                    n_iter=2,
                    n_jobs=-1,  
                    return_train_score=True,
                    refit=True, #use best model
                    #optimizer_kwargs={'base_estimator': 'GP'},
                    #random_state=42
                    )
# Running the optimization
opt.fit(X_train, y_train)

0:	learn: 0.9698025	total: 19ms	remaining: 2.83s
1:	learn: 0.9703492	total: 36.7ms	remaining: 2.71s
2:	learn: 0.9695678	total: 53.9ms	remaining: 2.64s
3:	learn: 0.9706443	total: 71.6ms	remaining: 2.61s
4:	learn: 0.9716385	total: 89.9ms	remaining: 2.61s
5:	learn: 0.9720353	total: 107ms	remaining: 2.57s
6:	learn: 0.9721974	total: 124ms	remaining: 2.53s
7:	learn: 0.9725820	total: 143ms	remaining: 2.53s
8:	learn: 0.9729787	total: 162ms	remaining: 2.53s
9:	learn: 0.9731432	total: 180ms	remaining: 2.52s
10:	learn: 0.9735303	total: 200ms	remaining: 2.52s
11:	learn: 0.9736682	total: 225ms	remaining: 2.59s
12:	learn: 0.9738883	total: 245ms	remaining: 2.58s
13:	learn: 0.9740843	total: 265ms	remaining: 2.57s
14:	learn: 0.9743165	total: 284ms	remaining: 2.55s
15:	learn: 0.9744302	total: 302ms	remaining: 2.53s
16:	learn: 0.9745584	total: 321ms	remaining: 2.51s
17:	learn: 0.9746334	total: 339ms	remaining: 2.48s
18:	learn: 0.9748100	total: 356ms	remaining: 2.45s
19:	learn: 0.9748680	total: 373ms	rema

BayesSearchCV(cv=None, error_score='raise',
              estimator=<catboost.core.CatBoostClassifier object at 0x7f94f3fd75c0>,
              fit_params=None, iid=True, n_iter=2, n_jobs=-1, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=True,
              scoring=make_scorer(roc_auc_score, needs_threshold=True),
              search_spaces={'depth': Integer(low=1, high=8, prior='uniform', transform='identity'),
                             'iterations': Integer(low=10, high=300, prior='uniform', transform='identity'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='identity')},
              verbose=0)

In [21]:
opt.best_params_

OrderedDict([('depth', 7),
             ('iterations', 150),
             ('learning_rate', 0.4178951456337519)])

In [24]:
print("Start Predicting")
predictions = opt.best_estimator_.predict(X_test)
pred_proba = opt.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9794
AUC : 0.921324
F1 Score : 0.604435


In [25]:
predictions = opt.best_estimator_.predict(test_data)

In [26]:
prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/catboost_bayesian_search_DATA.csv")

## CatBoost (Random Search)
#### Took 2 min
#### 0.72067 Accuracy on Kaggle 
#### 0.978 Accuracy on Test Set

In [31]:
parameters = {'depth'         : sp_randInt(1, 10), # 2 dk sürdü
              'learning_rate' : sp_randFloat(),
              'iterations'    : sp_randInt(10, 100)
}
model = CatBoostClassifier(task_type='GPU', eval_metric='Accuracy')
randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 2, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train, eval_set=(X_test,y_test))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0:	learn: 0.9693356	test: 0.9696662	best: 0.9696662 (0)	total: 28.3ms	remaining: 2.2s
1:	learn: 0.9710168	test: 0.9710265	best: 0.9710265 (1)	total: 55ms	remaining: 2.12s
2:	learn: 0.9725747	test: 0.9723643	best: 0.9723643 (2)	total: 81.5ms	remaining: 2.06s
3:	learn: 0.9732448	test: 0.9729626	best: 0.9729626 (3)	total: 107ms	remaining: 2s
4:	learn: 0.9738496	test: 0.9734763	best: 0.9734763 (4)	total: 133ms	remaining: 1.98s
5:	learn: 0.9744858	test: 0.9741254	best: 0.9741254 (5)	total: 159ms	remaining: 1.94s
6:	learn: 0.9749213	test: 0.9746560	best: 0.9746560 (6)	total: 185ms	remaining: 1.9s
7:	learn: 0.9751196	test: 0.9748761	best: 0.9748761 (7)	total: 209ms	remaining: 1.86s
8:	learn: 0.9757921	test: 0.9753559	best: 0.9753559 (8)	total: 243ms	remaining: 1.89s
9:	learn: 0.9760486	test: 0.9755026	best: 0.9755026 (9)	total: 269ms	remaining: 1.86s
10:	learn: 0.9763219	test: 0.9757962	best: 0.9757962 (10)	total: 295ms	remaining: 1.82s
11:	learn: 0.9765372	test: 0.9759824	best: 0.9759824 (11

RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7f87fe032710>,
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87fe032160>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87f24f4080>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87fe032320>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [32]:
randm.best_params_

{'depth': 7, 'iterations': 82, 'learning_rate': 0.8493356840525458}

In [33]:
print("Start Predicting")
predictions = randm.best_estimator_.predict(X_test)
pred_proba = randm.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))
print("Confusion Matrix: \n" ,metrics.confusion_matrix(y_test,predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9787
AUC : 0.913010
F1 Score : 0.603766
Confusion Matrix: 
 [[170525    512]
 [  3255   2870]]


In [34]:
predictions = randm.best_estimator_.predict(test_data)

In [35]:
prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/catboost_random_search_DATA.csv")

## XGBoost (Default Parameters)
#### Took 5 min
#### 0.6855 Accuracy on Kaggle 
#### 0.973 Accuracy on Test Set

In [13]:
model = XGBClassifier(predictor='gpu_predictor')

In [14]:
model.fit(X_train,y_train,eval_metric='auc') #5 dk

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic',
              predictor='gpu_predictor', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [16]:
print("Start Predicting")
predictions = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9731
AUC : 0.888142
F1 Score : 0.408346


In [17]:
predictions = model.predict(test_data)

In [18]:
prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/xgboost_default_parameters_DATA.csv")

## XGBoost (Bayesian Search)
#### Took 30 min ??
#### 0.7037 Accuracy on Kaggle 
#### 0.9753 Accuracy on Test Set

In [13]:
model = XGBClassifier(predictor= 'gpu_predictor') # 30 dk

# Converting average precision score into a scorer suitable for model selection
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
# Defining your search space
search_spaces = {'iterations': Integer(10, 300),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 #'random_strength': Real(1e-9, 10, 'log-uniform'),
                 #'bagging_temperature': Real(0.0, 1.0),
                 #'border_count': Integer(1, 255),
                 #'l2_leaf_reg': Integer(2, 30),
                 #'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                 }
# Setting up BayesSearchCV
opt = BayesSearchCV(model,
                    search_spaces,
                    scoring=roc_auc,
                    n_iter=2,
                    #n_jobs=-1,  
                    return_train_score=False,
                    refit=True, #use best model
                    optimizer_kwargs={'base_estimator': 'GP'},
                    )
# Running the optimization
opt.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

BayesSearchCV(cv=None, error_score='raise',
              estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                      colsample_bylevel=1, colsample_bynode=1,
                                      colsample_bytree=1, gamma=0,
                                      learning_rate=0.1, max_delta_step=0,
                                      max_depth=3, min_child_weight=1,
                                      missing=None, n_estimators=100, n_jobs=1,
                                      nthread=None, objective='binary:logistic',
                                      predictor='gpu_predictor', random_state=0...
              pre_dispatch='2*n_jobs', random_state=None, refit=True,
              return_train_score=False,
              scoring=make_scorer(roc_auc_score, needs_threshold=True),
              search_spaces={'depth': Integer(low=1, high=8, prior='uniform', transform='identity'),
                             'iterations': Integer(low=10, high=3

In [14]:
opt.best_params_

OrderedDict([('depth', 3),
             ('iterations', 254),
             ('learning_rate', 0.227369281198527)])

In [15]:
print("Start Predicting")
predictions = opt.best_estimator_.predict(X_test)
pred_proba = opt.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9753
AUC : 0.898982
F1 Score : 0.481859


In [16]:
predictions = opt.best_estimator_.predict(test_data)

In [17]:
prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/xgboost_bayesian_search_DATA.csv")

## XGBoost (Grid Search)
#### Took xx min 
#### 0.6883 Accuracy on Kaggle 
#### 0.xxxx Accuracy on Test Set

In [11]:
model = XGBClassifier(predictor= 'gpu_predictor')

param_test1 = {}
model = XGBClassifier()
gsearch1 = GridSearchCV(estimator=model,
                        param_grid=param_test1,
                        scoring='f1',
                        iid=False)

In [13]:
gsearch1.fit(X_train, y_train) # 19
print(gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'mean_fit_time': array([178.54620762]), 'std_fit_time': array([3.58291389]), 'mean_score_time': array([0.54726214]), 'std_score_time': array([0.03728819]), 'params': [{}], 'split0_test_score': array([0.4121372]), 'split1_test_score': array([0.40801068]), 'split2_test_score': array([0.40876683]), 'split3_test_score': array([0.42400211]), 'split4_test_score': array([0.42066225]), 'mean_test_score': array([0.41471582]), 'std_test_score': array([0.00645901]), 'rank_test_score': array([1], dtype=int32)} {} 0.4147158168717927


In [14]:
predictions = gsearch1.predict(test_data)

In [15]:
prediction_csv = pd.DataFrame(predictions, columns=['isFraud'])
indexes = [i for i in range(3663549,4170240)]
prediction_csv['TransactionID'] = indexes
prediction_csv.set_index('TransactionID', inplace=True)

prediction_csv.to_csv("/content/gdrive/MyDrive/inzva_week2_hw_dataset/predictions/xgboost_grid_search_DATA.csv")